# 基于MindSpore框架的MASS案例实现
## 1 模型简介
微软亚洲研究院于2019在ICML发表《MASS: Masked Sequence to Sequence Pre-training for Language Generation》，其借鑑了Bert的Masked Language Model预训练任务，提出了MAsked Sequence to Sequence Pre-training（MASS）模型，为自然语言生成任务联合预训练编码器和解码器。

MASS的编码器-解码器结构示例，图中“_”表示被屏蔽的词。
![](https://i.imgur.com/Jvhm0Dx.png)
编码器： 以被随机屏蔽掉连续片段的句子作为输入，BERT的做法是随机屏蔽掉15%的词，而MASS为了解决编码与解码之间的平衡，做法为屏蔽掉句子总长50%的片段。模型中使用特殊符号$[\mathbb M]$替换连续的单词来屏蔽片段，起始位置是随机的，且被选中的token有80%的概率是正常的$[\mathbb M]$token，10%的概率是被随机token替换，10%的概率保持原来的token。以上图为例，其中输入序列有8个单词，片段$x_3-x_6$被屏蔽掉。

解码器：输入为与编码器同样的序列，但是会屏蔽掉剩馀的词，然后解码器只预测编码器端屏蔽掉的词。以上图为例，只给定 $x_3x_4x_5$ 作为位置 4 - 6 的解码器输入，解码器会将 $[\mathbb M]$ 作为其他位置的输入（屏蔽了位置 1 − 3 和 7 − 8）。为了减少内存和计算成本，被屏蔽的token会被移除，未屏蔽token的位置编码不变（如果前两个标记被屏蔽并移除，第三个标记的位置仍然是 2 而不是 0)。通过这种方式，可以获得相似的准确度，并在解码器中减少 50% 的计算量。


```
encoder input (source): [x1, x2, x3, x4, x5, x6, x7, x8, </eos>]
masked encoder input:   [x1, x2, x3,  _,  _,  _, x7, x8, </eos>]
decoder input:          [  -, x3, x4, x5]
                          |   |   |   |
                          V   V   V   V
decoder output:         [x3, x4, x5, x6]
```

MASS预训练有以下几大优势：

(1) 编码器被强制去抽取未被屏蔽掉的词的含义，可以提升编码器理解源序列文本的能力。

(2) 通过在解码器端预测连续的标记，解码器可以比仅预测离散标记拥有更好的语言建模能力。

(3) 通过在解码器端进一步屏蔽在编码器端未被屏蔽掉的词， 以鼓励解码器从编码器端提取更多有用的信息来做预测，而不是依赖于前面预测出的单词，这样能促进编码器-解码器结构的联合训练。

### 1.1 模型结构

其模型基础结构可以使用任何Seq2Seq的结构，由于Transformer的优越性，故论文中使用Transformer模型作为基础结构，Transformer整体架构由 Encoder 和 Decoder 两个部分组成，不依赖任何RNN和CNN结构来生成输出，而是使用了Attention注意力机制，自动学习输入序列中每个单词和其他单词的关联，可以更好的处理长文本，且该模型可以高效的并行工作，训练速度较快。

Transformer 的整体架构如下：

<center>
<img src='https://i.imgur.com/ooO7ULP.png' height='600px'>
</center>

- 编码器和解码器分别由$N=6$个相同的编码器/解码器层组成。
- 在 Transformer 架构的左半部分，编码器的任务是将输入序列映射到一系列连续表示，然后将其馈送到解码器。
- 架构右半部分的解码器接收编码器的输出以及前一个时间步的解码器输出，以生成输出序列。
- 解码器的输出最终通过一个全连接层，然后是一个 softmax 层，以生成对输出序列下一个单词的预测。

### 1.2目标函数

给定一个未配对的源句子$x ∈ \mathcal X$，通过被屏蔽的序列$x^{\setminus u:v}$作为输入来预测句子片段$x^{u:v}$以预训练序列到序列模型。以极大似然函数作为目标函数：

$$
L(\theta; \mathcal X) = \frac{1}{ |\mathcal X|} \sum_{x\in \mathcal X} \log P(x^{u:v}|x^{\setminus u:v};\theta)\\
= \frac{1}{|\mathcal X|}\log \Pi_{t=u}^{v} P(x^{u:v}_t|x^{u:v}_{<t}, x^{\setminus u:v};\theta).
$$


注: $x^{u:v}$表示以句子位置$u$为起点$v$为终点的片段；$x^{\setminus u:v}$为$x^{u:v}$的修改版本，从$u$到$v$的片段被屏蔽，$0 < u < v < m$ 其中 $m$ 是句子 $x$ 长度。

### 1.3 模型特点
MASS 有一个重要的超参数 $k$，表示屏蔽的连续片段长度，通过调整 $k$ 的大小，MASS 能包含 BERT 中的掩码语言模型训练方法以及 GPT 中标准的语言模型预训练方法，使 MASS 成为一个通用的预训练框架。

当 $k = 1$ 时，根据MASS的设定，编码器端仅屏蔽一个单词，解码器以源序列中未屏蔽的单词为条件预测这个单词，如图(a)所示。由于解码器的所有输入都被屏蔽了，因此解码器本身就像一个非线性分类器，类似于 BERT 中使用的 softmax 矩阵。在这种情况下，条件概率为 $P (x^u|x^{\setminus u}; θ)$，$u$是掩码标记的位置，这正是 BERT3中使用的掩码语言模型的公式。

![](https://i.imgur.com/0QNrDSJ.png)

当 $k = m$（ $m$ 为序列长度）时，根据MASS的设定，编码器会屏蔽所有的单词，解码器需要预测所有单词，如图(b)所示。由于编码器端所有词都被屏蔽了，解码器的注意力机制相当于没有获取到信息，在这种情况下条件概率为 $P(x^{1:m}|x^{\setminus 1:m}; θ)$，等价于GPT中的标准语言模型。


![](https://i.imgur.com/2Q4pbLy.png)

## 2案例实现

### 2.1 環境建置
pip install -r requirements.txt

### 2.2 准备数据集
案例实现中预训练模型所使用的数据即News Crawl的英语单语数据数据集，下载好的数据集为一纯文字文件，接下来需要对该数据进行预处理，预处理包括对数据进行分词、利用subword-nmt工具做bpe编码、对分词后的语料应用该bpe编码并构建词彙表等工作。

而微调模型用于文本摘要任务所使用的数据集为Gigaword，该数据集已经有分割为训练、测试、验证集，有原文本(src)和目标摘要(tgt)两个文件，本案例只会使用训练及与测试集，数据集文件路径结构如下：

.Dataset/<br>
└── news_crawl<br>
&emsp;&emsp;└── news.2015.txt<br>
└── ggw_data<br>
&emsp;&emsp;├── test.src.txt<br>
&emsp;&emsp;├── test.tgt.txt<br>
&emsp;&emsp;├── train.src.txt<br>
&emsp;&emsp;└── train.tgt.txt<br>

In [1]:
"""對數據進行分詞"""
import os
from nltk.tokenize import word_tokenize

src_folder = "/Users/dawnkaslana/Workspace/Dataset/news_crawl/"
out_folder = "./tokenized_corpus/"

def create_tokenized_sentences(file_path, tokenized_file):
    tokenized_sen = []
    print(f" | Processing {file_path}.")
    with open(file_path, "r") as file:
        for sen in file:
            tokens = word_tokenize(sen)
            tokens = [t for t in tokens if t != " "]
            if len(tokens) > 175:
                continue
            tokenized_sen.append(" ".join(tokens) + "\n")

    with open(tokenized_file, "w") as file:
        file.writelines(tokenized_sen)
    print(f" | Wrote to {tokenized_file}.")

for file in os.listdir(src_folder):
    if not file.endswith(".txt"):
        continue
    file_path = os.path.join(src_folder, file)
    tokenized_file = os.path.join(out_folder, file.replace(".txt", "_tokenized.txt"))
    create_tokenized_sentences(file_path, tokenized_file)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/dawnkaslana/Workspace/Dataset/news_crawl/'

In [ ]:
"""利用subword-nmt工具生成bpe檔案"""
src_folder_path = '/Users/dawnkaslana/Workspace/Dataset/news_crawl/' # source text folder path.
os.system("cd %s && cat *.txt | subword-nmt learn-bpe -s 46000 -o all.bpe.codes" % (src_folder_path))

In [ ]:
"""應用該bpe檔案並構建詞彙表."""
from src.utils import Dictionary
import subprocess

source_folder = os.path.abspath("./tokenized_corpus/")
output_folder = os.path.abspath("./tokenized_corpus/bpe/")
codes = os.path.abspath("./all.bpe.codes")
vocab_path = "./vocab/vocab_en.dict.bin"

ENCODER = "subword-nmt apply-bpe -c"
LEARN_DICT = "subword-nmt get-vocab -i"
def bpe_encode(codes_path, src_path, output_path, dict_path):
    # Encoding.
    print(" | Applying BPE encoding.")
    commands = ENCODER.split() + [codes_path] + ["-i"] + [src_path] + ["-o"] + [output_path]
    subprocess.call(commands)
    print(" | Fetching vocabulary from single file.")
    # Learn vocab.
    commands = LEARN_DICT.split() + [output_path] + ["-o"] + [dict_path]
    subprocess.call(commands)

available_dict = []
for file in os.listdir(source_folder):
    if file.endswith(".txt"):
        output_path = os.path.join(output_folder, file.replace(".txt", "_bpe.txt"))
        dict_path = os.path.join(output_folder, file.replace(".txt", ".dict"))
        available_dict.append(dict_path)
        bpe_encode(codes, os.path.join(source_folder, file), output_path, dict_path)

# 加载bpe_encode處理過的文本词汇表，行格式为word frequency。
vocab = Dictionary.load_from_text(available_dict)
vocab.persistence(vocab_path) #将词汇表对象保存为二进制文件。
print(f" | Vocabulary Size: {len(vocab)}")

### 2.2 生成NewsCrawl數據集

In [13]:
"""Create News Crawl Pre-Training Dataset."""
import os
from src.dataset import MonoLingualDataLoader
from src.language_model import LooseMaskedLanguageModel
from src.utils import Dictionary

input_folder_path = '/Users/dawnkaslana/Workspace/Dataset/news_crawl/'
output_folder_path = './train_data/news_crawl_dataset/'
vocab_path = './vocab/vocab_en.dict.bin'

def create_pre_train(text_file, max_sen_len):
    vocab = Dictionary.load_from_persisted_dict(vocab_path)

    loader = MonoLingualDataLoader(
        src_filepath=text_file,
        lang="en", dictionary=vocab,
        language_model=LooseMaskedLanguageModel(mask_ratio=0.4, mask_all_prob=None),
        max_sen_len=max_sen_len, min_sen_len=10
    )

    src_file_name = os.path.basename(text_file)

    file_name = os.path.join(
        output_folder_path,
        src_file_name.replace('.txt', f'_len_{max_sen_len}.tfrecord')
    )
    loader.write_to_tfrecord(path=file_name)

for file in os.listdir(input_folder_path):
    if file.endswith(".txt"):
        create_pre_train(os.path.join(input_folder_path, file), 32)

print(f" | Generate Dataset for Pre-training is done.")
print(f" | Vocabulary size: {vocab.size}.")

 | Processing corpus /Users/dawnkaslana/Workspace/Dataset/news_crawl/news2007.txt.
 | Shortest len = 1.
 | Longest  len = 3269.
 | Total    sen = 2573547.
 | Write to /Users/dawnkaslana/Workspace/HWmodels/official/nlp/mass/train_data/news_crawl_dataset/news2007_len_32.tfrecord-001-of-001.
 | Generate Dataset for Pre-training is done.
 | Vocabulary size: 353.


### 2.2 生成Gigaword數據集

In [3]:
"""Generate Gigaword dataset."""
import os
from src.dataset import BiLingualDataLoader
from src.language_model import NoiseChannelLanguageModel
from src.utils import Dictionary

input_folder_path = '/Users/dawnkaslana/Workspace/Dataset/ggw_data/'
output_folder_path = './train_data/gigaword_dataset/'
vocab_path = './vocab/vocab_en.dict.bin'

vocab = Dictionary.load_from_persisted_dict(vocab_path)

train = BiLingualDataLoader(
    src_filepath=os.path.join(input_folder_path,"train.src.txt"),
    tgt_filepath=os.path.join(input_folder_path,"train.tgt.txt"),
    src_dict=vocab, tgt_dict=vocab,
    src_lang="en", tgt_lang="en",
    language_model=NoiseChannelLanguageModel(add_noise_prob=0.),
    max_sen_len=32
)

train.write_to_tfrecord(
    path=os.path.join(output_folder_path, "gigaword_train_dataset.tfrecord")
)

test = BiLingualDataLoader(
    src_filepath=os.path.join(input_folder_path,"test.src.txt"),
    tgt_filepath=os.path.join(input_folder_path,"test.tgt.txt"),
    src_dict=vocab, tgt_dict=vocab,
    src_lang="en", tgt_lang="en",
    language_model=NoiseChannelLanguageModel(add_noise_prob=0),
    max_sen_len=32
)

test.write_to_tfrecord(
    path=os.path.join(output_folder_path, "gigaword_test_dataset.tfrecord")
)

print(f" | Generate Dataset for fine-tuneing is done.")
print(f" | Vocabulary size: {vocab.size}.")

 | Processing corpus /Users/dawnkaslana/Workspace/Dataset/ggw_data/org_data/train.src.txt.
 | Processing corpus /Users/dawnkaslana/Workspace/Dataset/ggw_data/org_data/train.tgt.txt.
 | Shortest len = 3.
 | Longest  len = 100.
 | Total    sen = 1981314.
 | Total token num=87383811, 87.10378401784284% replaced by <unk>.
 | Write to /Users/dawnkaslana/Workspace/HWmodels/official/nlp/mass/train_data/gigaword_dataset/gigaword_train_dataset.tfrecord-001-of-001.
 | Processing corpus /Users/dawnkaslana/Workspace/Dataset/ggw_data/org_data/test.src.txt.
 | Processing corpus /Users/dawnkaslana/Workspace/Dataset/ggw_data/org_data/test.tgt.txt.
 | Shortest len = 2.
 | Longest  len = 73.
 | Total    sen = 1081.
 | Total token num=46933, 85.92248524492362% replaced by <unk>.
 | Write to /Users/dawnkaslana/Workspace/HWmodels/official/nlp/mass/train_data/gigaword_dataset/gigaword_test_dataset.tfrecord-001-of-001.
 | Generate Dataset for fine-tuneing is done.
 | Vocabulary size: 353.


## 預訓練

bash run_ascend.sh -t t -n 1 -i 1 <br>
python train.py --device_target Ascend --output_path './output'


## 微调

bash run_ascend.sh -t t -n 1 -i 1

## 推理

bash run_ascend.sh -t i -n 1 -i 1 -o {outputfile}

MindSpore version:  1.7.0
The result of multiplication calculation is correct, MindSpore has been installed successfully!
